In [1]:
# Import Dependencies
import pandas as pd
import re
import spacy
import json
from spacy.attrs import ORTH, LEMMA, NORM, TAG
# Load spacy model
nlp = spacy.load('en_core_web_sm',parser=False, entity=False) 


In [2]:
# Replace happy and sad emoticons with words "happy" and "sad"
# from: https://towardsdatascience.com/extracting-twitter-data-pre-processing-and-sentiment-analysis-using-python-3-0-7192bd8b47cf)
def emoticons_replace(tweet):
    
    emoticons ={'Happy': [
        ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
        ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
        '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
        'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
        '<3'
        ], 
                'Sad': [
        ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
        ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
        ':c', ':{', '>:\\', ';('
        ]}
    a=tweet
    temp=""
    # for word in a.split():
    #      print (emoticons.get(word, word))

    converted = []
    for word in a.split():
        new_word = word
        for emotion_key, emotion_val in emoticons.items():
            if word in emotion_val:
                new_word = emotion_key
                break
        converted.append(new_word)


    converted = ' '.join(converted)
        
    return(converted)
    

In [3]:
def Limitizing(tweet,nlp):
    TOKENIZER_EXCEPTIONS = {
    # do
        "don't": [
            {ORTH: "do", LEMMA: "do"},
            {ORTH: "n't", LEMMA: "not", NORM: "not", TAG: "RB"}],
        "doesn't": [
            {ORTH: "does", LEMMA: "do"},
            {ORTH: "n't", LEMMA: "not", NORM: "not", TAG: "RB"}],
        "didn't": [
            {ORTH: "did", LEMMA: "do"},
            {ORTH: "n't", LEMMA: "not", NORM: "not", TAG: "RB"}],
    # can
        "can't": [
            {ORTH: "ca", LEMMA: "can"},
            {ORTH: "n't", LEMMA: "not", NORM: "not", TAG: "RB"}],
        "couldn't": [
            {ORTH: "could", LEMMA: "can"},
            {ORTH: "n't", LEMMA: "not", NORM: "not", TAG: "RB"}],
    # have
        "I've'": [
            {ORTH: "I", LEMMA: "I"},
            {ORTH: "'ve'", LEMMA: "have", NORM: "have", TAG: "VERB"}],
        "haven't": [
            {ORTH: "have", LEMMA: "have"},
            {ORTH: "n't", LEMMA: "not", NORM: "not", TAG: "RB"}],
        "hasn't": [
            {ORTH: "has", LEMMA: "have"},
            {ORTH: "n't", LEMMA: "not", NORM: "not", TAG: "RB"}],
        "hadn't": [
            {ORTH: "had", LEMMA: "have"},
            {ORTH: "n't", LEMMA: "not", NORM: "not", TAG: "RB"}],
    # will/shall will be replaced by will
        "I'll'": [
            {ORTH: "I", LEMMA: "I"},
            {ORTH: "'ll'", LEMMA: "will", NORM: "will", TAG: "VERB"}],
        "he'll'": [
            {ORTH: "he", LEMMA: "he"},
            {ORTH: "'ll'", LEMMA: "will", NORM: "will", TAG: "VERB"}],
        "she'll'": [
            {ORTH: "she", LEMMA: "she"},
            {ORTH: "'ll'", LEMMA: "will", NORM: "will", TAG: "VERB"}],
        "it'll'": [
            {ORTH: "it", LEMMA: "it"},
            {ORTH: "'ll'", LEMMA: "will", NORM: "will", TAG: "VERB"}],
        "won't": [
            {ORTH: "wo", LEMMA: "will"},
            {ORTH: "n't", LEMMA: "not", NORM: "not", TAG: "RB"}],
        "wouldn't": [
            {ORTH: "would", LEMMA: "will"},
            {ORTH: "n't", LEMMA: "not", NORM: "not", TAG: "RB"}],
    # be
        "I'm'": [
            {ORTH: "I", LEMMA: "I"},
            {ORTH: "'m'", LEMMA: "be", NORM: "am", TAG: "VERB"}]
    }
    for tok, rule in TOKENIZER_EXCEPTIONS.items():
            nlp.tokenizer.add_special_case('u'+tok, rule)

#     nlp = spacy.load('en_core_web_sm',parser=False, entity=False) 


    # New stop words list 
    nlp.Defaults.stop_words -= {'but', 'again', 'five', 'front','keep', 'by', 'with', 'nothing', 'be', 'make', 'on', 'can','down', 
                                'against','can','above', 'become','nor','serious', 'have', 'should','not','third','latter', 'never', 'becomes', 'doing', 'did', 'sometimes', 'since', 'to', 'show', 'across', "n't", 'itself', 'such',
                                'formerly', 'how', 'part', 'thereby', 'wherever', 'done', 'bottom', 'least', 'alone', 'below',
                                'former', 'made','first', 'top','being', 'up', 'one', 'neither', 'without', 'became', 'empty',"will"
                                'over', 'no', 'well'}

    nlp.Defaults.stop_words |= {"-PRON-",}



    stops = nlp.Defaults.stop_words
    # print (stops)
    doc=nlp(tweet)
    temp=[]
    for token in doc:
        word = token.lemma_
        if word not in nlp.Defaults.stop_words:
            temp.append(token.lemma_)


    temp = ' '.join(temp)
    return(temp)

In [18]:
# Load the language model and parse your document.
# nlp =spacy.load('en_core_web_sm')  # en for English; others available.
# tweet = "This is a sample @sentence, I didn't & make it. $AAPL"

def process_tweet(tweet,nlp):
    # Replace emotions
    tweet=emoticons_replace(tweet)
    # Replace emoji
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+','', tweet)
    # Remove hyperlinks
    tweet = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))\S+","",tweet)
    # To lowercase
    tweet = tweet.lower()

  
    # Remove HTML special entities (e.g. &amp;) 
    tweet = re.sub(r"\&\w*;"," ", tweet)
    #Convert @username to AT_USER 
    tweet = re.sub(r"@[^\s]+","",tweet)

#     # Remove hashtags
#     tweet = re.sub(r'#\w*', ' ', tweet)

    # Remove new line characters)
    tweet = re.sub(r'\n\n+', ' ', tweet)
    # Process Limitization using spacy
    tweet= Limitizing(tweet,nlp)
    # Remove sepcial characters
    tweet = re.sub(r"[!#$%&'\\()*+,-./:;<=>?@\^_`{|}~]", " ", tweet)
    tweet = re.sub(r"\[", " ", tweet)
    tweet = re.sub(r"\]", " ", tweet)
    # replace any leading and trailing whitespaces including tabs (\t).
#     tweet=tweet.strip()
#     tweet=tweet.lstrip()
    # Remove whitespace (not including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    tweet=" ".join(tweet.split())
    return(tweet)

# clean_tweet=process_tweet(tweet)


# print (clean_tweet)



## Import Dataset

In [19]:
twitter_df = pd.read_csv("data/BidenTrain.csv", error_bad_lines=False)

In [20]:
text="test string "
twitter_df=twitter_df.iloc[:10]

In [21]:
#  calls function process_tweet to clean the twitter_df
# warning
nlp = spacy.load('en_core_web_sm',parser=False, entity=False) 
twitter_df["Text"] = twitter_df["Text"].map(lambda x: process_tweet(x,nlp))

In [22]:
# Decided not to replace text abbreviations with full words b/c the abbreviations carry their own meaning
# Decided to keep hashtags other than the # b/c they may carry their own meaning
twitter_df.head()

,Date,Favorite,ID,Retweet,Text
0,2019-07-31 13:20:35,0,1156555255473160194,0,not like point not like i want to make look st...
1,2019-07-31 13:20:32,0,1156555245700468742,0,disingenuous medicareforall joe
2,2019-07-31 13:20:27,0,1156555221054758912,0,tonight need not to attack but turn answer to ...
3,2019-07-31 13:20:25,0,1156555213710471168,0,look logo national twin day be offer instead l...
4,2019-07-31 13:20:18,0,1156555183385694209,0,trump refuse to help country people be flee pr...


In [129]:
from datetime import datetime, timedelta

twitter_df["Date"] = twitter_df["Date"].map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S') - timedelta(hours=4))

In [106]:
twitter_df.head()

,Date,Favorite,ID,Retweet,Text
0,2019-07-31 09:20:35,0,1156555255473160194,0,not like point not like i want to make look s...
1,2019-07-31 09:20:32,0,1156555245700468742,0,disingenuous medicareforall joe
2,2019-07-31 09:20:27,0,1156555221054758912,0,tonight need not to attack but turn answer t...
3,2019-07-31 09:20:25,0,1156555213710471168,0,look logo national twin day be offer instead ...
4,2019-07-31 09:20:18,0,1156555183385694209,0,trump refuse to help country people be flee ...


In [109]:
import sqlite3

# def main():

    
    # Connect to database and drop/insert "tweeter_poll" table if exists  ---run once only
 # create a database connection

conn = sqlite3.connect("politics_db.db")

c = conn.cursor()


def Create_teble(c):
# Create table twitter

    c.execute("DROP TABLE IF EXISTS twitter;")
    c.execute("""CREATE TABLE "twitter" (
            "id" INTEGER UNIQUE,
            "tweet" TEXT, 
            "tweet_date" TEXT,
            "retweet" INTEGER,
            "favorite" INTEGER,
            "candidate" TEXT,
            "sentiment" INTEGER     
            )""")


    c.execute("DROP TABLE IF EXISTS all_candidate;")
    c.execute("""CREATE TABLE "all_candidate" (
            "candidate" TEXT UNIQUE,
            "first_name" TEXT, 
            "last_name" TEXT           
            )""")

Create_teble(c)


In [112]:
def Insert_row(conn,rown):
    
    sql = ''' INSERT INTO all_candidate(candidate,first_name,last_name)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, rown)
    return cur.lastrowid


def all_candidate_rows(conn):
    
    row1=('@Biden','Joe','Biden')
    row2=('@Warren','Elizabeth','Warren')
    
    Insert_row(conn, row1)
    Insert_row(conn, row2)
    

In [113]:
all_candidate_rows(conn)
twitter_df.rename(
        columns={
            "ID": "id",
            "Text":"tweet",
            "Date":"tweet_date",
            "Retweet":"retweet",
            "Favorite":"favorite"},inplace=True)
#     twitter_df.to_sql("tweeter_poll", conn, if_exists="append")
twitter_df.head()

,tweet_date,favorite,id,retweet,tweet
0,2019-07-31 09:20:35,0,1156555255473160194,0,not like point not like i want to make look s...
1,2019-07-31 09:20:32,0,1156555245700468742,0,disingenuous medicareforall joe
2,2019-07-31 09:20:27,0,1156555221054758912,0,tonight need not to attack but turn answer t...
3,2019-07-31 09:20:25,0,1156555213710471168,0,look logo national twin day be offer instead ...
4,2019-07-31 09:20:18,0,1156555183385694209,0,trump refuse to help country people be flee ...


In [114]:
twitter_df["candidate"]="@Biden"
twitter_df["sentiment"]="" 

In [115]:

twitter_df=twitter_df.set_index('id')
twitter_df.head()

,tweet_date,favorite,retweet,tweet,candidate,sentiment
id,,,,,,
1156555255473160194,2019-07-31 09:20:35,0,0,not like point not like i want to make look s...,@Biden,
1156555245700468742,2019-07-31 09:20:32,0,0,disingenuous medicareforall joe,@Biden,
1156555221054758912,2019-07-31 09:20:27,0,0,tonight need not to attack but turn answer t...,@Biden,
1156555213710471168,2019-07-31 09:20:25,0,0,look logo national twin day be offer instead ...,@Biden,
1156555183385694209,2019-07-31 09:20:18,0,0,trump refuse to help country people be flee ...,@Biden,


In [116]:
twitter_df.head()

,tweet_date,favorite,retweet,tweet,candidate,sentiment
id,,,,,,
1156555255473160194,2019-07-31 09:20:35,0,0,not like point not like i want to make look s...,@Biden,
1156555245700468742,2019-07-31 09:20:32,0,0,disingenuous medicareforall joe,@Biden,
1156555221054758912,2019-07-31 09:20:27,0,0,tonight need not to attack but turn answer t...,@Biden,
1156555213710471168,2019-07-31 09:20:25,0,0,look logo national twin day be offer instead ...,@Biden,
1156555183385694209,2019-07-31 09:20:18,0,0,trump refuse to help country people be flee ...,@Biden,


In [117]:
twitter_df.to_sql("twitter", conn, if_exists="append")


In [119]:
# table = sql.read_frame('select * from twitter', con)
# table.to_csv('date/test.csv')

In [80]:
# data = conn.execute("SELECT * FROM twitter").fetchall()

In [83]:
# data()

(1156555245700468742,
 '  disingenuous   medicareforall joe',
 '2019-07-31 09:20:32',
 0,
 0,
 '@Biden',
 '')

## Text Vectorization

## Split into testing and training datasets

### Cross-validation?
What cross-validation does is splitting the training data into a certain number of training folds 
(with 75% of the training data) and a the same number of testing folds (with 25% of the training data), 
use the training folds to train the classifier, and test it against the testing folds to obtain performance 
metrics (see below). The process is repeated multiple times and an average for each of the metrics is calculated.

If your testing set is always the same, you might be overfitting to that testing set, which means you might 
be adjusting your analysis to a given set of data so much that you might fail to analyze a different set. 
Cross-validation helps prevent that.
The more data you have, the more folds you will be able to use.

## Apply algorithm (1 to begin)